# 第5章: 係り受け解析
---
夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

In [94]:
!head -25 neko.txt.cabocha

* 0 -1D 0/0 0.000000
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
* 0 2D 0/0 -0.764522
　	記号,空白,*,*,*,*,　,　,　
* 1 2D 0/1 -0.764522
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 2 -1D 0/2 0.000000
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS
* 0 2D 0/1 -1.911675
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/0 -1.911675
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
* 2 -1D 0/0 0.000000
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。
EOS
EOS



## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラス`Morph`を実装せよ．このクラスは表層形（`surface`），基本形（`base`），品詞（`pos`），品詞細分類1（`pos1`）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文を`Morph`オブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [6]:
%%file q40.py
from itertools import groupby
import sys
from string import printable

class Morph:
    """cabocha lattice formatファイルの1行を読み込む"""
    __slots__ = ['surface', 'pos', 'pos1', 'base']
    exceptions = frozenset(printable)
    # 吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
    def __init__(self, line):
        self.surface, temp = line.rstrip().split('\t')
        inf = temp.split(',')
        self.pos = inf[0]
        self.pos1 = inf[1]
        # 4章同様, 半角文字の基本形問題
        if self.surface in self.exceptions:
            self.base = self.surface
        else:
            self.base = inf[6]
    
    
    def __str__(self):
        return self.surface
    
    def __repr__(self):
        return 'q40.Morph({})'.format(', '.join((self.surface, self.pos, self.pos1, self.base)))
    

def main():
    for i, sent_lis in enumerate(load_cabocha(sys.stdin), start=1):
        if i == 3:
            print(*sent_lis)
            for word in sent_lis:
                print(repr(word))

            
def load_cabocha(fi):
    """cabocha lattice formatファイルからMorphインスタンスを生成"""
    for is_eos, sentence in groupby(fi, key=lambda x: x == 'EOS\n'):
        if not is_eos:
            yield [Morph(line) for line in sentence if not line.startswith('* ')]
            # startswith('*')だと表層形が「*」のときにまずい

if __name__ == '__main__':
    main()


Overwriting q40.py


In [7]:
!python q40.py < neko.txt.cabocha

名前 は まだ 無い 。
q40.Morph(名前, 名詞, 一般, 名前)
q40.Morph(は, 助詞, 係助詞, は)
q40.Morph(まだ, 副詞, 助詞類接続, まだ)
q40.Morph(無い, 形容詞, 自立, 無い)
q40.Morph(。, 記号, 句点, 。)


In [89]:
import q40
q40.Morph('"	名詞,サ変接続,*,*,*,*,*')

q40.Morph(", 名詞, サ変接続, ")

In [90]:
q40.Morph(',	記号,読点,*,*,*,*,",",",",",",,')

q40.Morph(,, 記号, 読点, ,)

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [1]:
%%file q41.py
import sys
from itertools import groupby

from q40 import Morph


class Chunk:
    """cabocha lattice formatファイルから分節を読み込む。q40.Morphをヘルパーとして利用。"""
    
    __slots__ = ['idx', 'dst', 'morphs', 'srcs']
    # * 0 2D 0/0 -0.764522
    def __init__(self, line):
        info = line.rstrip().split()
        self.idx = int(info[1])
        self.dst = int(info[2].rstrip("D"))
        self.morphs = []
        self.srcs = []
    
    def __str__(self):
        return ''.join([morph.surface for morph in self.morphs])
    
    def __repr__(self):
        return 'q41.Chunk({}, {})'.format(self.idx, self.dst)
    
    def srcs_append(self, src_idx):
        """係り元文節インデックスを追加"""
        self.srcs.append(src_idx)
    
    def morphs_append(self, line):
        """形態素を追加"""
        self.morphs.append(Morph(line))
    
    def chunk2str(self):
        """句読点を取り除いた文節の表層形を返す"""
        return ''.join([morph.surface for morph in self.morphs if morph.pos != '記号'])
    
    def contain_pos(self, pos):
        """文節中にある品詞が存在するかどうかを返す"""
        return pos in (morph.pos for morph in self.morphs)

class Sentence:
    """cabocha lattice formatファイルから文を読み込む。Chunkクラスをヘルパーとして利用。"""
    __slots__ = ['chunks', 'idx']
    
    def __init__(self, sent_lines):
        self.chunks = []
        ch_append = self.chunks.append
        for line in sent_lines:                    
            if line.startswith('* '):
                ch_append(Chunk(line))
            else:
                self.chunks[-1].morphs_append(line)
        
        for chunk in self.chunks:
            if chunk.dst != -1:
                self.chunks[chunk.dst].srcs_append(chunk.idx)
    
    def __str__(self):
        return ' '.join([morph.surface for chunk in self.chunks for morph in chunk.morphs])
    
    def print_dep_idx(self):
        """係り元文節インデックスと係り先文節インデックスを表示"""
        for chunk in self.chunks:
            print('{}:{} => {}'.format(chunk.idx, chunk, chunk.dst))
    
    def print_dep(self):
        """係り元文節と係り先文節の表層をタブ区切りで表示"""
        for chunk in self.chunks:
            print('{}\t{}'.format(chunk.chunk2str(), self.chunks[chunk.dst].chunk2str()))
            
    def print_noun_verb_dep(self):
        """名詞を含む文節が動詞を含む文節に係るものを抽出"""
        for chunk in self.chunks:
            if chunk.contain_pos('名詞') and self.chunks[chunk.dst].contain_pos('動詞'):
                print('{}\t{}'.format(chunk.chunk2str(), self.chunks[chunk.dst].chunk2str()))
            


def main():
    for i, sent in enumerate(cabocha2sentences(sys.stdin), start=1):
        if i == 8:
            sent.print_dep_idx()

def cabocha2sentences(fi):
    """cabocha lattice formatファイルからSentenceインスタンスを生成"""
    for is_eos, sentence in groupby(fi, key=lambda x: x == 'EOS\n'):
        if not is_eos:
            yield (Sentence(sentence))


if __name__ == '__main__':
    main()


Overwriting q41.py


In [8]:
!python q41.py < neko.txt.cabocha

0:この => 1
1:書生というのは => 7
2:時々 => 4
3:我々を => 4
4:捕えて => 5
5:煮て => 6
6:食うという => 7
7:話である。 => -1


## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [5]:
%%file q42.py
import sys


from q41 import Sentence, cabocha2sentences

def main():
    for i, sent in enumerate(cabocha2sentences(sys.stdin), start=1):
        if i < 5:
            sent.print_dep()
        else:
            break

if __name__ == '__main__':
    main()


Overwriting q42.py


In [6]:
!python q42.py < neko.txt.cabocha

一	一
	猫である
吾輩は	猫である
猫である	猫である
名前は	無い
まだ	無い
無い	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ
つかぬ	つかぬ


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [7]:
%%file q43.py
import sys


from q41 import Sentence, cabocha2sentences

def main():
    for i, sent in enumerate(cabocha2sentences(sys.stdin), start=1):
        if i < 10:
            sent.print_noun_verb_dep()
        else:
            break

if __name__ == '__main__':
    main()

Overwriting q43.py


In [8]:
!python q43.py < neko.txt.cabocha

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
記憶している	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて


In [ ]:
%%timeit
'printable